<a href="https://colab.research.google.com/github/Tristan-Brown1096/DS18_Unit_2_Build_Week_Project/blob/master/unit_2_build_week_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data wrangling

In [ ]:
import pandas as pd
import numpy as np
!pip install category_encoders==2.*
!pip install pandas-profiling==2.*

In [2]:
df = pd.read_csv('cbb_data.csv')

In [3]:
df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
0,North Carolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
1,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
2,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
3,Texas Tech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
4,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [4]:
df.YEAR = df.YEAR.astype(str)

In [5]:
df.TEAM = [team.replace(' ', '') for team in df.TEAM]

In [6]:
df['ID'] = df.TEAM + df.YEAR

In [7]:
df = df.set_index('ID')

In [8]:
df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
ID,,,,,,,,,,,,,,,,,,,,,,,,
NorthCarolina2016,NorthCarolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1.0,2016
Wisconsin2015,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1.0,2015
Michigan2018,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3.0,2018
TexasTech2019,TexasTech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3.0,2019
Gonzaga2017,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1.0,2017


In [9]:
df.POSTSEASON = df.POSTSEASON.replace(np.NaN, 'no tourney')

In [10]:
df.SEED = df.SEED.replace(np.NaN, 0)

In [ ]:
df.SEED = df.SEED.astype(int)

In [12]:
df.head()

,TEAM,CONF,G,W,ADJOE,ADJDE,BARTHAG,EFG_O,EFG_D,TOR,TORD,ORB,DRB,FTR,FTRD,2P_O,2P_D,3P_O,3P_D,ADJ_T,WAB,POSTSEASON,SEED,YEAR
ID,,,,,,,,,,,,,,,,,,,,,,,,
NorthCarolina2016,NorthCarolina,ACC,40,33,123.3,94.9,0.9531,52.6,48.1,15.4,18.2,40.7,30.0,32.3,30.4,53.9,44.6,32.7,36.2,71.7,8.6,2ND,1,2016
Wisconsin2015,Wisconsin,B10,40,36,129.1,93.6,0.9758,54.8,47.7,12.4,15.8,32.1,23.7,36.2,22.4,54.8,44.7,36.5,37.5,59.3,11.3,2ND,1,2015
Michigan2018,Michigan,B10,40,33,114.4,90.4,0.9375,53.9,47.7,14.0,19.5,25.5,24.9,30.7,30.0,54.7,46.8,35.2,33.2,65.9,6.9,2ND,3,2018
TexasTech2019,TexasTech,B12,38,31,115.2,85.2,0.9696,53.5,43.0,17.7,22.8,27.4,28.7,32.9,36.6,52.8,41.9,36.5,29.7,67.5,7.0,2ND,3,2019
Gonzaga2017,Gonzaga,WCC,39,37,117.8,86.3,0.9728,56.6,41.1,16.2,17.1,30.0,26.2,39.0,26.9,56.3,40.0,38.2,29.0,71.5,7.7,2ND,1,2017


#Split data

In [13]:
y = df.POSTSEASON

X = df.drop(['POSTSEASON', 'TEAM', 'YEAR', 'SEED'], axis=1)

In [14]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

#Baseline

In [16]:
baseline = y_train.value_counts(normalize=True).max()
print('Baseline Accuracy:', baseline)

Baseline Accuracy: 0.797864768683274


#Build model_1

In [21]:
from category_encoders import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from category_encoders import OneHotEncoder
from sklearn.pipeline import make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest

In [18]:
model_1 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_depth=4)
)

model_1.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['CONF'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'CONF',
                                          'data_type': dtype('O'),
                                          'mapping': Amer     1
MAAC     2
CUSA     3
SEC      4
BSky     5
WCC      6
SB       7
BE       8
A10      9
P12     10
WAC     11
MWC     12
ACC     13
B10     14
MAC     15
B12     16
NEC     17
SWAC    18
AE      19
MVC     20
CAA     21
Slnd    22
Horz    23
OVC     24
Sum     25
Pat     26
SC      27
ASun    28
Ivy     29
MEAC    30
BSth    31...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=4, max_features='auto',
                                        max_leaf_no

In [19]:
print('Training Accuracy:', model_1.score(X_train, y_train))
print('Validation Accuracy:', model_1.score(X_val, y_val))

Training Accuracy: 0.8725978647686833
Validation Accuracy: 0.8863636363636364


#Tuning

In [20]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

In [ ]:
model_2 = make_pipeline(
    OrdinalEncoder(),
    SelectKBest(),
    RandomForestClassifier()
)

params = {'selectkbest__k': range(1, 20, 2),
          'randomforestclassifier__n_estimators': range(100, 500, 20),
          'randomforestclassifier__max_depth': range(1, 20, 2),
}

gs = GridSearchCV(model_2, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [ ]:
gs.fit(X_train, y_train)

Fitting 5 folds for each of 2000 candidates, totalling 10000 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:    8.4s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:   38.9s
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed:  4.6min
[Parallel(n_jobs=-1)]: Done 1246 tasks      | elapsed:  7.6min
[Parallel(n_jobs=-1)]: Done 1796 tasks      | elapsed: 12.1min
[Parallel(n_jobs=-1)]: Done 2446 tasks      | elapsed: 17.0min
[Parallel(n_jobs=-1)]: Done 3196 tasks      | elapsed: 24.2min
[Parallel(n_jobs=-1)]: Done 4046 tasks      | elapsed: 33.0min
[Parallel(n_jobs=-1)]: Done 4996 tasks      | elapsed: 42.8min
[Parallel(n_jobs=-1)]: Done 6046 tasks      | elapsed: 53.2min
[Parallel(n_jobs=-1)]: Done 7196 tasks      | elapsed: 64.1min
[Parallel(n_jobs=-1)]: Done 8446 tasks      | elapsed: 76.2min
[Parallel(n_jobs=-1)]: Done 9796 tasks      | elapsed: 90.5min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('ordinalencoder',
                                        OrdinalEncoder(cols=None,
                                                       drop_invariant=False,
                                                       handle_missing='value',
                                                       handle_unknown='value',
                                                       mapping=None,
                                                       return_df=True,
                                                       verbose=0)),
                                       ('selectkbest',
                                        SelectKBest(k=10,
                                                    score_func=<function f_classif at 0x7fe4d07778c8>)),
                                       ('randomforestclassifier',
                                        RandomForestClassifie...
   

In [ ]:
gs.best_params_

{'randomforestclassifier__max_depth': 7,
 'randomforestclassifier__n_estimators': 320,
 'selectkbest__k': 5}

#Build model_2

In [ ]:
model_2 = gs.best_estimator_

In [ ]:
print('Training accuracy:', model_2.score(X_train, y_train))
print('Validation accuracy:', model_2.score(X_val, y_val))

Training accuracy: 0.9323843416370107
Validation accuracy: 0.8693181818181818


#More tuning

In [23]:
params = {'randomforestclassifier__n_estimators': [320],
          'randomforestclassifier__max_depth': range(2, 22, 2),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_1, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [24]:
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 30 candidates, totalling 150 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   43.4s
[Parallel(n_jobs=-1)]: Done 150 out of 150 | elapsed:  3.0min finished


{'randomforestclassifier__max_depth': 6,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__n_estimators': 320}

#Build model_3

In [26]:
model_3 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=320, max_depth=6, max_features=None)
)

model_3.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['CONF'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'CONF',
                                          'data_type': dtype('O'),
                                          'mapping': Amer     1
MAAC     2
CUSA     3
SEC      4
BSky     5
WCC      6
SB       7
BE       8
A10      9
P12     10
WAC     11
MWC     12
ACC     13
B10     14
MAC     15
B12     16
NEC     17
SWAC    18
AE      19
MVC     20
CAA     21
Slnd    22
Horz    23
OVC     24
Sum     25
Pat     26
SC      27
ASun    28
Ivy     29
MEAC    30
BSth    31...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=6, max_features=None,
                                        max_leaf_node

In [27]:
print('Training accuracy:', model_3.score(X_train, y_train))
print('Validation accuracy:', model_3.score(X_val, y_val))

Training accuracy: 0.9323843416370107
Validation accuracy: 0.875


#Tuning again

In [29]:
params = {'randomforestclassifier__n_estimators': range(320, 420, 10),
          'randomforestclassifier__max_depth': range(4, 12),
          'randomforestclassifier__max_features': ['sqrt', 'log2', None]
}

gs = GridSearchCV(model_3, param_grid=params, n_jobs=-1, verbose=1, cv=5, scoring='accuracy')

In [30]:
gs.fit(X_train, y_train)
gs.best_params_

Fitting 5 folds for each of 240 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:   32.6s
[Parallel(n_jobs=-1)]: Done 196 tasks      | elapsed:  3.1min
[Parallel(n_jobs=-1)]: Done 446 tasks      | elapsed:  8.3min
[Parallel(n_jobs=-1)]: Done 796 tasks      | elapsed: 15.7min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed: 25.5min finished


{'randomforestclassifier__max_depth': 5,
 'randomforestclassifier__max_features': None,
 'randomforestclassifier__n_estimators': 320}

#Build model_4

In [31]:
model_4 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=320, max_depth=5, max_features=None)
)

model_4.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['CONF'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'CONF',
                                          'data_type': dtype('O'),
                                          'mapping': Amer     1
MAAC     2
CUSA     3
SEC      4
BSky     5
WCC      6
SB       7
BE       8
A10      9
P12     10
WAC     11
MWC     12
ACC     13
B10     14
MAC     15
B12     16
NEC     17
SWAC    18
AE      19
MVC     20
CAA     21
Slnd    22
Horz    23
OVC     24
Sum     25
Pat     26
SC      27
ASun    28
Ivy     29
MEAC    30
BSth    31...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=5, max_features=None,
                                        max_leaf_node

In [32]:
print('Training accuracy:', model_4.score(X_train, y_train))
print('Validation accuracy:', model_4.score(X_val, y_val))

Training accuracy: 0.892526690391459
Validation accuracy: 0.8778409090909091


#Build model_5

In [41]:
model_5 = make_pipeline(
    OrdinalEncoder(),
    SimpleImputer(),
    RandomForestClassifier(random_state=42, n_estimators=500, max_depth=5, max_features=None)
)

model_5.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('ordinalencoder',
                 OrdinalEncoder(cols=['CONF'], drop_invariant=False,
                                handle_missing='value', handle_unknown='value',
                                mapping=[{'col': 'CONF',
                                          'data_type': dtype('O'),
                                          'mapping': Amer     1
MAAC     2
CUSA     3
SEC      4
BSky     5
WCC      6
SB       7
BE       8
A10      9
P12     10
WAC     11
MWC     12
ACC     13
B10     14
MAC     15
B12     16
NEC     17
SWAC    18
AE      19
MVC     20
CAA     21
Slnd    22
Horz    23
OVC     24
Sum     25
Pat     26
SC      27
ASun    28
Ivy     29
MEAC    30
BSth    31...
                 RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                                        max_depth=5, max_features=None,
                                        max_leaf_node

In [42]:
print('Training accuracy:', model_5.score(X_train, y_train))
print('Validation accuracy:', model_5.score(X_val, y_val))

Training accuracy: 0.8932384341637011
Validation accuracy: 0.875
